## Loading the libraries

**NOTE**: Since I didn't have access to a GPU, I used Kaggle's free GPU to run this. And since it was taking a long time and breaking in the middle, I had to find a few fixes. Therefore, by the end of it, I could only run it once, so I have only generated the embeddings for the upregulation of _C9orf72_.

In [1]:
pip install anndata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 58.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install helical

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 52.0 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of gtfparse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of gtfparse to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtr

In [3]:
print("Modules installed correctly, starting the run!")

Modules installed correctly, starting the run!


In [4]:
import os
import gc
import torch
import numpy as np
import anndata as ad
from scipy import sparse
from helical.models.geneformer import Geneformer, GeneformerConfig

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

INFO:datasets:PyTorch version 2.6.0+cu124 available.
INFO:datasets:Polars version 1.21.0 available.
INFO:datasets:Duckdb version 1.2.2 available.
INFO:datasets:TensorFlow version 2.18.0 available.
INFO:datasets:JAX version 0.5.2 available.
2025-09-29 09:13:42.693555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759137222.869226      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759137222.922480      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


## Importing Geneformer using Helical

Here I am simply loading the Geneformer model based off of Helical's documentation.

In [5]:
model_config = GeneformerConfig(model_name="gf-12L-95M-i4096", 
                                device=device, 
                                batch_size=10)

geneformer_v2 = Geneformer(model_config)

INFO:helical.utils.downloader:Downloading 'geneformer/v2/gene_median_dictionary.pkl'
INFO:helical.utils.downloader:Starting to download: 'https://helicalpackage.s3.eu-west-2.amazonaws.com/geneformer/v2/gene_median_dictionary.pkl'
gene_median_dictionary.pkl: 100%|██████████| 1.51M/1.51M [00:01<00:00, 821kB/s]
INFO:helical.utils.downloader:File saved to: '/root/.cache/helical/models/geneformer/v2/gene_median_dictionary.pkl'
INFO:helical.utils.downloader:Downloading 'geneformer/v2/token_dictionary.pkl'
INFO:helical.utils.downloader:Starting to download: 'https://helicalpackage.s3.eu-west-2.amazonaws.com/geneformer/v2/token_dictionary.pkl'
token_dictionary.pkl: 100%|██████████| 426k/426k [00:00<00:00, 207MB/s]
INFO:helical.utils.downloader:File saved to: '/root/.cache/helical/models/geneformer/v2/token_dictionary.pkl'
INFO:helical.utils.downloader:Downloading 'geneformer/v2/ensembl_mapping_dict.pkl'
INFO:helical.utils.downloader:Starting to download: 'https://helicalpackage.s3.eu-west-2.am

This is a simple function to run Geneformer. It does the following:

- Reads in the data.
- Run Geneformer
- Gets the embeddings
- And prints the shape while returning the embeddings. 

This function is super simple and makes it a little cleaner to run the code. 

In [6]:
def run_geneformer(h5ad_path):
    ann_data = ad.read_h5ad(h5ad_path)
    dataset = geneformer_v2.process_data(ann_data, use_raw_counts = True)
    embeddings = geneformer_v2.get_embeddings(dataset)
    print("Base model embeddings shape:", embeddings.shape)
    return embeddings

## Generating the embeddings

Here I am simply running Geneformer and getting the embeddings. And I am saving those embeddings to be exported off of Kaggle so that I can analyze them locally. 

In [ ]:
base_embeddings = run_geneformer("/kaggle/input/helical-compressed/adata_base_5k.h5ad")
np.save("/kaggle/working/emb_base_5k.npy", base_embeddings)

INFO:helical.models.geneformer.model:Processing data for Geneformer.
INFO:pyensembl.download_cache:Fetching /root/.cache/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.110.gtf.gz from URL https://ftp.ensembl.org/pub/release-110/gtf/homo_sapiens/Homo_sapiens.GRCh38.110.gtf.gz
INFO:datacache.download:Downloading https://ftp.ensembl.org/pub/release-110/gtf/homo_sapiens/Homo_sapiens.GRCh38.110.gtf.gz to /root/.cache/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.110.gtf.gz
INFO:pyensembl.download_cache:Fetching /root/.cache/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL https://ftp.ensembl.org/pub/release-110/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
INFO:datacache.download:Downloading https://ftp.ensembl.org/pub/release-110/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz to /root/.cache/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.cdna.all.fa.gz
INFO:pyensembl.download_cache:Fetching /root/.cache/pyensembl/GRCh38/ensembl110/Homo_sa

  0%|          | 0/11202 [00:00<?, ?it/s]

https://pmc.ncbi.nlm.nih.gov/articles/PMC6417666/

In [ ]:
C9orf72_ALS_up_embeddings = run_geneformer("/kaggle/input/helical-compressed/adata_C9orf72_ALS_up_5k.h5ad")
np.save("/kaggle/working/emb_C9orf72_ALS_up_5k.npy", C9orf72_ALS_up_embeddings)